In [23]:
# @title 1. 🟢 Initialize & Connect Hardware { display-mode: "form" }
# @markdown ### **Option A: Cloud Mode (Default)**
# @markdown 1. Simply click **Play**. It will use Google's free T4 GPU.
# @markdown
# @markdown ---
# @markdown ### **Option B: Local Mode (Turbo - For Your RTX 4080)**
# @markdown 1. **Setup Apps:** Install [Ollama](https://ollama.com) and [Node.js](https://nodejs.org).
# @markdown 2. **Pull AI Model:** Open PowerShell and run: `ollama pull llama3.1`
# @markdown 3. **Sync Local Python:** Run this in PowerShell:
# @markdown    `python -m pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121 --upgrade; python -m pip install jupyter_server jupyter_http_over_ws notebook ollama jupyter_client streamlit ytmusicapi duckduckgo_search --upgrade`
# @markdown 4. **Start Bridge:** In PowerShell, run:
# @markdown    `python -m notebook --NotebookApp.allow_origin='https://colab.research.google.com' --port=8888`
# @markdown 5. **Connect Colab:** Top right arrow > **Connect to local runtime** > Paste the `localhost:8888` URL.

import os, subprocess, time, platform, sys

is_local = "COLAB_GPU" not in os.environ
print(f"🛠️ Verifying Environment Support (OS: {platform.system()})...")

if is_local:
    subprocess.run([sys.executable, "-m", "pip", "install", "ytmusicapi", "streamlit", "ollama", "torch", "jupyter_client", "duckduckgo_search", "-q"])
else:
    !pip install ytmusicapi ollama duckduckgo_search -q
    if not os.path.exists("/usr/local/bin/ollama"):
        !curl -fsSL https://ollama.com/install.sh | sh > /dev/null 2>&1

import torch, ollama
has_gpu = torch.cuda.is_available()
print(f"⚡ Mode: {'LOCAL' if is_local else 'CLOUD'} | 🚀 GPU: {torch.cuda.get_device_name(0) if has_gpu else 'CPU'}")

if is_local:
    try:
        models = [m['name'] for m in ollama.list()['models']]
        if 'llama3.1:latest' in models: print("✅ Local Llama 3.1 detected.")
        else: print("⚠️ ACTION: Run 'ollama pull llama3.1' in PowerShell.")
    except: print("⚠️ ACTION: Open the Ollama app on Windows Taskbar!")
else:
    subprocess.Popen(["/usr/local/bin/ollama", "serve"], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    time.sleep(5)
    subprocess.run(["/usr/local/bin/ollama", "pull", "llama3.1"], stdout=subprocess.DEVNULL)

print("\n✅ Initialization Complete. Proceed to Step 2.")

🛠️ Verifying Environment Support (OS: Windows)...
⚡ Mode: LOCAL | 🚀 GPU: NVIDIA GeForce RTX 4080
⚠️ ACTION: Open the Ollama app on Windows Taskbar!

✅ Initialization Complete. Proceed to Step 2.


In [37]:
# @title 2. 🚀 [STEP 2] Launch Studio App { display-mode: "form" }
import urllib.request, subprocess, time, os, sys

try:
    external_ip = urllib.request.urlopen('https://ident.me').read().decode('utf8')
except:
    external_ip = "Local-Link"

with open("app.py", "w", encoding="utf-8") as f:
    f.write(f"""
import streamlit as st
import pandas as pd
import ollama
import os
import re
from ytmusicapi import YTMusic
from duckduckgo_search import DDGS

st.set_page_config(page_title="K-Pop Studio Pro", layout="wide", page_icon="💎")

# --- UI STYLING ---
st.markdown(\"\"\"
    <style>
    .tier-container {{ padding: 20px; border-radius: 15px; margin-bottom: 25px; border: 1px solid rgba(255,255,255,0.1); }}
    .tier-PEAK {{ background: linear-gradient(135deg, #FFD70022 0%, #000000 100%); border-left: 5px solid #FFD700 !important; }}
    .tier-SSS {{ background: linear-gradient(135deg, #9D50BB22 0%, #000000 100%); border-left: 5px solid #9D50BB !important; }}
    .tier-S {{ background: linear-gradient(135deg, #ff4b4b22 0%, #000000 100%); border-left: 5px solid #ff4b4b !important; }}
    .tier-A {{ background: linear-gradient(135deg, #00C9FF22 0%, #000000 100%); border-left: 5px solid #00C9FF !important; }}
    .tier-B {{ background: linear-gradient(135deg, #92FE9D22 0%, #000000 100%); border-left: 5px solid #92FE9D !important; }}
    .tier-C {{ background: linear-gradient(135deg, #bdc3c722 0%, #000000 100%); border-left: 5px solid #bdc3c7 !important; }}

    .group-card-designer {{
        display: flex; flex-direction: column; align-items: center;
        background: rgba(0,0,0,0.3); padding: 12px; border-radius: 15px;
        border: 1px solid rgba(255,255,255,0.1); transition: 0.2s;
    }}
    .logo-designer {{ width: 70px; height: 70px; border-radius: 50%; object-fit: cover; border: 2px solid rgba(255,255,255,0.5); margin-bottom: 8px; }}
    .stat-badge {{ background: rgba(255, 75, 75, 0.15); color: #ff4b4b; padding: 3px 12px; border-radius: 50px; font-size: 0.75rem; font-weight: 800; border: 1px solid rgba(255, 75, 75, 0.3); }}
    </style>
    \"\"\", unsafe_allow_html=True)

# --- DEFAULTS ---
if 'hardcoded_order' not in st.session_state:
    st.session_state.hardcoded_order = [
        'aespa', 'MEOVV', 'BabyMonster', 'ILLIT', 'STAYC', 'IVE',
        'tripleS', 'Kep1er', 'izna', 'NMIXX', 'LE SSERAFIM', 'H//PE Princess',
        'BLACKPINK', 'ITZY', 'Red Velvet', 'H1-KEY', 'FIFTY FIFTY', 'baby DONT cry',
        'NewJeans', 'Billlie', 'Kiiikiii', 'Hearts2Hearts', 'QWER', 'RESCENE',
        'ifeye', 'KIIRAS', 'ARTMS', 'fromis_9', 'TWICE', 'BADVILLAIN',
        'I-DLE', 'KISS OF LIFE', 'VVS', 'VIVIZ', 'AtHeart', 'PURPLE KISS',
        'MAMAMOO', 'CLC', 'EVERGLOW', 'XG', 'KATSEYE', 'TRI.BE', 'YOUNG POSSE'
    ]

if 'tiers' not in st.session_state:
    st.session_state.tiers = {{gp: 'PEAK' if i < 6 else 'SSS' if i == 6 else 'S' if i < 15 else 'A' if i < 27 else 'B' if i < 35 else 'C' for i, gp in enumerate(st.session_state.hardcoded_order)}}

TIER_INFO = {{
    'PEAK': {{'label': '🏆 PEAK', 'color': '#FFD700'}},
    'SSS': {{'label': '💎 SSS', 'color': '#9D50BB'}},
    'S': {{'label': '🔥 S-Tier', 'color': '#ff4b4b'}},
    'A': {{'label': '⭐ A-Tier', 'color': '#00C9FF'}},
    'B': {{'label': '📈 B-Tier', 'color': '#92FE9D'}},
    'C': {{'label': '💤 C-Tier', 'color': '#bdc3c7'}}
}}

if 'baked_data' not in st.session_state: st.session_state.baked_data = pd.DataFrame()

@st.cache_data(ttl=604800)
def fetch_logo(name):
    try:
        yt = YTMusic()
        search = yt.search(name, filter="artists")
        return search[0]['thumbnails'][-1]['url'] if search else "https://via.placeholder.com/150"
    except: return "https://via.placeholder.com/150"

@st.dialog("🎥 Player")
def play_video(vid_url):
    st.video(vid_url)

@st.dialog("🎯 Move & Shuffle")
def move_group(gp_name):
    st.write(f"Adjusting **{{gp_name}}**")
    new_tier = st.selectbox("Tier", list(TIER_INFO.keys()), index=list(TIER_INFO.keys()).index(st.session_state.tiers[gp_name]))

    # Position logic
    current_idx = st.session_state.hardcoded_order.index(gp_name)
    new_pos = st.number_input("Order Position", 0, len(st.session_state.hardcoded_order)-1, current_idx)

    if st.button("Update Position & Tier"):
        st.session_state.tiers[gp_name] = new_tier
        # Move in list
        st.session_state.hardcoded_order.remove(gp_name)
        st.session_state.hardcoded_order.insert(new_pos, gp_name)
        st.rerun()

t1, t2 = st.tabs(["🎛️ Tier Designer", "🚀 Discovery Hub"])

with t1:
    st.write("### ⚡ Studio Controls")
    c1, c2, _ = st.columns([1, 2, 4])
    target_year = c1.text_input("Analysis Year", "2026", label_visibility="collapsed")
    if c2.button("🔥 BAKE DATA", use_container_width=True):
        yt = YTMusic()
        results = []
        bar = st.progress(0, text="📡 Strict Metadata Scan...")
        for i, gp in enumerate(st.session_state.hardcoded_order):
            bar.progress((i+1)/len(st.session_state.hardcoded_order), text=f"🔍 Verifying {{gp}} 2026 Releases...")
            try:
                # METHOD: Browse Artist's Singles specifically to verify Year metadata
                search = yt.search(gp, filter="artists")
                if search:
                    artist_id = search[0]['browseId']
                    artist_data = yt.get_artist(artist_id)
                    for section in ['singles', 'albums']:
                        if section in artist_data and 'results' in artist_data[section]:
                            for item in artist_data[section]['results']:
                                # FETCH FULL ALBUM TO GET VERIFIED YEAR
                                alb = yt.get_album(item['browseId'])
                                if str(alb.get('year')) == str(target_year):
                                    for track in alb.get('tracks', []):
                                        results.append({{
                                            'Group': gp, 'Tier': st.session_state.tiers[gp], 'Song': track['title'],
                                            'vid': track['videoId'], 'Views': '2026 Metadata Match', 'Year': alb['year']
                                        }})
            except: continue
        st.session_state.baked_data = pd.DataFrame(results).drop_duplicates('vid').reset_index(drop=True)
        st.success(f"Bake Complete! Found {{len(st.session_state.baked_data)}} verified tracks.")

    st.divider()
    for tier_key, info in TIER_INFO.items():
        st.markdown(f'<div class="tier-container tier-{{tier_key}}"><h3 style="color:{{info["color"]}};">{{info["label"]}}</h3></div>', unsafe_allow_html=True)
        # FILTER GROUPS BY TIER AND MAINTAIN THE SHUFFLED ORDER
        tier_groups = [gp for gp in st.session_state.hardcoded_order if st.session_state.tiers[gp] == tier_key]
        if tier_groups:
            cols = st.columns(8)
            for i, gp in enumerate(tier_groups):
                with cols[i % 8]:
                    img = fetch_logo(gp)
                    st.markdown(f'''<div class="group-card-designer"><img src="{{img}}" class="logo-designer"><div style="font-size:0.75rem; font-weight:bold; text-align:center;">{{gp}}</div></div>''', unsafe_allow_html=True)
                    if st.button("Move", key=f"m_{{gp}}"): move_group(gp)
        else: st.caption("Empty Tier.")

with t2:
    if not st.session_state.baked_data.empty:
        st.sidebar.header("⚙️ View Settings")
        sort_choice = st.sidebar.selectbox("Sort Global List By:", ["Tier Rank (Default)", "Alphabetical (Group)"])
        df = st.session_state.baked_data

        if sort_choice == "Alphabetical (Group)": df = df.sort_values(by='Group')
        else:
            df['Tier_Val'] = df['Tier'].map({{k: v for v, k in enumerate(TIER_INFO.keys())}})
            df = df.sort_values(by='Tier_Val')

        for idx in df.index:
            row = df.loc[idx]
            c_info, c_play, c_ai, _ = st.columns([4, 1, 1, 2])
            with c_info: st.markdown(f'<div style="display:flex;align-items:center;"><span style="font-weight:700;">{{row["Group"]}} - {{row["Song"]}}</span><span class="stat-badge">📅 {{row["Year"]}}</span></div>', unsafe_allow_html=True)
            if c_play.button("▶️ Watch", key=f"p_{{idx}}"): play_video(f"https://www.youtube.com/watch?v={{row['vid']}}")
            if c_ai.button("🧠 AI", key=f"ai_{{idx}}"):
                resp = ollama.chat(model='llama3.1', messages=[{{'role': 'user', 'content': f"Analysis for {{row['Song']}} by {{row['Group']}}" }}])
                st.session_state.baked_data.at[idx, 'Analysis'] = resp['message']['content']
                st.rerun()
            if row.get('Analysis'): st.info(row['Analysis'])
            st.divider()
    else:
        st.info("No verified 2026 data. Use 'Bake' to scan official discographies.")
""")

# Launch
print(f"\n🔑 TUNNEL PASSWORD: {external_ip}")
subprocess.Popen([sys.executable, "-m", "streamlit", "run", "app.py", "--server.port", "8501", "--server.headless", "true"], start_new_session=True)
time.sleep(5)
print("🌐 Studio Active: http://localhost:8501")
!npx localtunnel --port 8501


🔑 TUNNEL PASSWORD: 136.27.10.153
🌐 Studio Active: http://localhost:8501


C:\Users\Karman\AppData\Local\npm-cache\_npx\75ac80b86e83d4a2\node_modules\localtunnel\bin\lt.js:81
    throw err;
    ^

Error: connection refused: localtunnel.me:20635 (check your firewall settings)
    at Socket.<anonymous> (C:\Users\Karman\AppData\Local\npm-cache\_npx\75ac80b86e83d4a2\node_modules\localtunnel\lib\TunnelCluster.js:52:11)
    at Socket.emit (node:events:508:28)
    at emitErrorNT (node:internal/streams/destroy:170:8)
    at emitErrorCloseNT (node:internal/streams/destroy:129:3)
    at process.processTicksAndRejections (node:internal/process/task_queues:89:21)

Node.js v24.13.1


your url is: https://odd-spies-appear.loca.lt
^C
